In [6]:
#!pip install geopandas pyogrio

In [1]:
import os
import math
import geopandas as gpd
from shapely.geometry import box
from shapely import affinity
import fiona

# ================= CONFIGURATION =================
# ================= CONFIGURATION =================
# INPUT_GDB = r'C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot24\New File Geodatabase.gdb'    # Path to input GDB
# OUTPUT_GDB = r'C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot24\CorrectedPlotDimensions.gdb'  # Path to output GDB (will be created if missing)

# INPUT_GDB = r'C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\plot2022\New File Geodatabase.gdb'    # Path to input GDB
# OUTPUT_GDB = r'C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\plot2022\CorrectedPlotDimensions.gdb'  # Path to output GDB (will be created if missing)

INPUT_GDB = r'C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\New File Geodatabase.gdb'    # Path to input GDB
OUTPUT_GDB = r'C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\25plots_geometry.gdb'  # Path to output GDB (will be created if missing)


#TARGET_LENGTH = 6.1
TARGET_LENGTH = 5.15
#TARGET_WIDTH = 0.94
TARGET_WIDTH = 1
# =================================================


def get_orientation_angle(geom):
    """
    Calculates the angle of the longest side of the minimum rotated rectangle.
    """
    rect = geom.minimum_rotated_rectangle
    if rect.geom_type != 'Polygon':
        return 0

    coords = list(rect.exterior.coords)
    p0, p1, p2 = coords[0], coords[1], coords[2]
    
    # Calculate distances to find the "long" side
    dist01 = math.sqrt((p1[0]-p0[0])**2 + (p1[1]-p0[1])**2)
    dist12 = math.sqrt((p2[0]-p1[0])**2 + (p2[1]-p1[1])**2)
    
    if dist01 > dist12:
        dx, dy = p1[0] - p0[0], p1[1] - p0[1]
    else:
        dx, dy = p2[0] - p1[0], p2[1] - p1[1]
        
    angle_rad = math.atan2(dy, dx)
    return math.degrees(angle_rad)

def correct_geometry(geom, length, width):
    """
    Creates a standardized box centered at the original centroid 
    and rotated to match the original orientation.
    """
    if geom is None or geom.is_empty:
        return geom

    centroid = geom.centroid
    angle = get_orientation_angle(geom)
    
    # Create box centered at 0,0
    half_l, half_w = length / 2, width / 2
    new_shape = box(-half_l, -half_w, half_l, half_w)
    
    # Rotate
    new_shape = affinity.rotate(new_shape, angle, origin=(0, 0))
    
    # Translate to original position
    new_shape = affinity.translate(new_shape, xoff=centroid.x, yoff=centroid.y)
    
    return new_shape

def main():
    # 1. Get list of layers
    try:
        layers = fiona.listlayers(INPUT_GDB)
        print(f"Found {len(layers)} layers in input GDB.")
    except Exception as e:
        print(f"Error checking input GDB: {e}")
        return

    for layer_name in layers:
        print(f"Processing layer: {layer_name}...")
        
        try:
            # Read the layer (Attributes AND Geometry are loaded by default)
            gdf = gpd.read_file(INPUT_GDB, layer=layer_name, engine='pyogrio')
            
            # CRS Check
            if gdf.crs and gdf.crs.is_geographic:
                print(f"  WARNING: Layer {layer_name} is in Lat/Lon. Dimensions will be wrong!")

            # Apply Geometry Correction
            # This updates the geometry column while keeping all other attribute columns intact
            gdf['geometry'] = gdf['geometry'].apply(
                lambda g: correct_geometry(g, TARGET_LENGTH, TARGET_WIDTH)
            )
            
            # Save to Output GDB
            # We save the whole 'gdf', so attributes are preserved automatically
            gdf.to_file(
                OUTPUT_GDB, 
                layer=layer_name, 
                driver="OpenFileGDB", 
                engine='pyogrio'
            )
            print(f"  Saved with attributes to {OUTPUT_GDB}")
            
        except Exception as e:
            print(f"  Failed to process {layer_name}: {e}")

if __name__ == "__main__":
    main()

Found 11 layers in input GDB.
Processing layer: SEVREC2501...


C:\Users\bazrafka\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field id of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
C:\Users\bazrafka\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field id of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(


  Saved with attributes to C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\25plots_geometry.gdb
Processing layer: SEVREC2502...
  Saved with attributes to C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\25plots_geometry.gdb
Processing layer: SEVREC2503...
  Saved with attributes to C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\25plots_geometry.gdb
Processing layer: SEVREC2504...


C:\Users\bazrafka\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field id of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
C:\Users\bazrafka\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field id of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
C:\Users\bazrafka\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field id of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(


  Saved with attributes to C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\25plots_geometry.gdb
Processing layer: SEVREC2505...
  Saved with attributes to C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\25plots_geometry.gdb
Processing layer: SEVREC2506...


C:\Users\bazrafka\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field id of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
C:\Users\bazrafka\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field id of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
C:\Users\bazrafka\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field id of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(


  Saved with attributes to C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\25plots_geometry.gdb
Processing layer: SEVREC2507...
  Saved with attributes to C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\25plots_geometry.gdb
Processing layer: SEVREC2508...
  Saved with attributes to C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\25plots_geometry.gdb
Processing layer: SEVREC2509...


C:\Users\bazrafka\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field id of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
C:\Users\bazrafka\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field id of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(


  Saved with attributes to C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\25plots_geometry.gdb
Processing layer: SEVREC2510...
  Saved with attributes to C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\25plots_geometry.gdb
Processing layer: SEVREC2512...
  Saved with attributes to C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\25plots_geometry.gdb


C:\Users\bazrafka\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field id of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
